In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import pickle
%matplotlib inline

In [13]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

geolocator = Nominatim(user_agent="my_user_agent")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=0.1)

town_closest_distances = pd.read_pickle("town_closest_distances.pkl")

In [14]:
town_closest_distances.head()

,Town,latitude,longitude,closest_solid_waste,closest_renewable_energy,closest_petroleum_release,closest_pollution_reports
0,Abingdon town,36.71,-81.97,0.294818,52.129728,0.042871,0.460100
1,Accomac town,37.72,-75.67,6.249408,1.550936,0.015395,0.058127
2,Adwolf,36.79,-81.59,5.154079,30.339459,0.433561,4.275153
3,Alberta town,36.86,-77.89,6.861292,19.785872,0.339769,13.516906
4,Alexandria city,38.82,-77.09,0.811872,25.651648,0.133541,0.811872


In [17]:
town_closest_distances["country"] = town_closest_distances.apply(lambda row : geolocator.reverse(str(row.latitude) + " , " + str(row.longitude)), axis=1)

In [29]:
town_closest_distances.rename(columns={"country" : "full_address"}, inplace=True)

In [33]:
town_closest_distances["full_address"] = town_closest_distances["full_address"].apply(lambda location : location.address)

In [41]:
town_closest_distances["full_address"][1].split(",")[4].split(" ")[1]
#.split(" ")[0]

'Accomack'

In [55]:
town_closest_distances["full_address"][4]

'3520, Aspinwall Lane, Douglas MacArthur, Chapel Hill, Alexandria, Virginia, 22304, United States'

In [77]:
town_closest_distances["county"] = town_closest_distances["full_address"].apply(lambda address : ("".join([s for s in address.split(",") if "County" in s])))

In [78]:
town_closest_distances.head()

,Town,latitude,longitude,closest_solid_waste,closest_renewable_energy,closest_petroleum_release,closest_pollution_reports,full_address,county
0,Abingdon town,36.71,-81.97,0.294818,52.129728,0.042871,0.460100,"162, A Street Southeast, Abingdon, Washington ...",Washington County
1,Accomac town,37.72,-75.67,6.249408,1.550936,0.015395,0.058127,Accomack-Northampton Planning District Commiss...,Accomack County
2,Adwolf,36.79,-81.59,5.154079,30.339459,0.433561,4.275153,"1031, Grubmore Road, Adwolf, Smyth County, Vir...",Smyth County
3,Alberta town,36.86,-77.89,6.861292,19.785872,0.339769,13.516906,"Branch Drive, Alberta, Brunswick County, Virgi...",Brunswick County
4,Alexandria city,38.82,-77.09,0.811872,25.651648,0.133541,0.811872,"3520, Aspinwall Lane, Douglas MacArthur, Chape...",


In [91]:
town_closest_distances_filtered = town_closest_distances[(town_closest_distances["county"].astype(str).str.len() >= 1) & (town_closest_distances["county"].astype(str).str.len() <= 25)]

In [92]:
town_closest_distances_filtered.head()

,Town,latitude,longitude,closest_solid_waste,closest_renewable_energy,closest_petroleum_release,closest_pollution_reports,full_address,county
0,Abingdon town,36.71,-81.97,0.294818,52.129728,0.042871,0.460100,"162, A Street Southeast, Abingdon, Washington ...",Washington County
1,Accomac town,37.72,-75.67,6.249408,1.550936,0.015395,0.058127,Accomack-Northampton Planning District Commiss...,Accomack County
2,Adwolf,36.79,-81.59,5.154079,30.339459,0.433561,4.275153,"1031, Grubmore Road, Adwolf, Smyth County, Vir...",Smyth County
3,Alberta town,36.86,-77.89,6.861292,19.785872,0.339769,13.516906,"Branch Drive, Alberta, Brunswick County, Virgi...",Brunswick County
5,Allisonia,36.93,-80.73,2.436219,18.531150,2.281144,3.835388,"Irish Mountain Road, Pulaski County, Virginia,...",Pulaski County


In [93]:
town_closest_distances_filtered["county"].apply(lambda x : print(x))

 Washington County
 Accomack County
 Smyth County
 Brunswick County
 Pulaski County
 Campbell County
 Amelia County
 Amherst County
 Fairfax County
 Wise County
 Warren County
 Appomattox County
 Stafford County
 Loudoun County
 Arlington County
 Nelson County
 Loudoun County
 Hanover County
 Smyth County
 Accomack County
 Augusta County
 Fairfax County
 Mecklenburg County
 Henry County
 Shenandoah County
 Accomack County
 Fauquier County
 Bedford County
 Fairfax County
 Accomack County
 Chesterfield County
 Loudoun County
 Rockingham County
 Pulaski County
 Montgomery County
 Isle of Wight County
 Chesterfield County
 Clarke County
 York County
 Bedford County
 Wise County
 Montgomery County
 Nottoway County
 Bland County
 Accomack County
 Botetourt County
 Louisa County
 Tazewell County
 Accomack County
 Chesterfield County
 Franklin County
 Accomack County
 Stafford County
 Caroline County
 Clarke County
 Mecklenburg County
 Southampton County
 Mecklenburg County
 Loudoun County
 So

0      None
1      None
2      None
3      None
5      None
       ... 
587    None
588    None
589    None
590    None
591    None
Name: county, Length: 550, dtype: object

In [94]:
town_closest_distances_filtered["county"] = town_closest_distances_filtered["county"].apply(lambda x : x.split(" ")[1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [96]:
town_closest_distances_filtered["county"].apply(lambda x : print(x))

Washington
Accomack
Smyth
Brunswick
Pulaski
Campbell
Amelia
Amherst
Fairfax
Wise
Warren
Appomattox
Stafford
Loudoun
Arlington
Nelson
Loudoun
Hanover
Smyth
Accomack
Augusta
Fairfax
Mecklenburg
Henry
Shenandoah
Accomack
Fauquier
Bedford
Fairfax
Accomack
Chesterfield
Loudoun
Rockingham
Pulaski
Montgomery
Isle
Chesterfield
Clarke
York
Bedford
Wise
Montgomery
Nottoway
Bland
Accomack
Botetourt
Louisa
Tazewell
Accomack
Chesterfield
Franklin
Accomack
Stafford
Caroline
Clarke
Mecklenburg
Southampton
Mecklenburg
Loudoun
Southampton
Chesterfield
Rockingham
Madison
Loudoun
Rockingham
Brunswick
Campbell
Botetourt
Prince
Buckingham
Prince
Prince
Fairfax
Fairfax
Nottoway
Alleghany
County
Isle
Carroll
Northampton
Southampton
Accomack
Isle
Isle
Loudoun
Russell
Fauquier
Accomack
Roanoke
Tazewell
King
Fairfax
Henrico
Fairfax
Charles
Charlotte
Mecklenburg
Accomack
Pittsylvania
Henry
Northampton
William
Chesterfield
Rappahannock
Smyth
Accomack
Montgomery
Augusta
Surry
Mecklenburg
Tazewell
Russell
Alleghany

0      None
1      None
2      None
3      None
5      None
       ... 
587    None
588    None
589    None
590    None
591    None
Name: county, Length: 550, dtype: object

In [97]:
town_closest_distances_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 550 entries, 0 to 591
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Town                       550 non-null    object 
 1   latitude                   550 non-null    float64
 2   longitude                  550 non-null    float64
 3   closest_solid_waste        550 non-null    float64
 4   closest_renewable_energy   550 non-null    float64
 5   closest_petroleum_release  550 non-null    float64
 6   closest_pollution_reports  550 non-null    float64
 7   full_address               550 non-null    object 
 8   county                     550 non-null    object 
dtypes: float64(6), object(3)
memory usage: 43.0+ KB


In [100]:
county_socioeconomic = pd.read_csv("CountySocioeconomic.csv")

In [101]:
county_socioeconomic.head()

,county,median income,bachelors degree,white,black,hispanic,asian
0,Accomack,"43,210",18.5,68.1,28.6,9.3,0.8
1,Albemarle,"75,394",53.5,81.6,9.7,5.9,5.6
2,Alexandria city,"96,733",62.1,62.0,21.9,16.8,6.1
3,Alleghany,"47,794",15.4,92.7,4.7,1.5,0.3
4,Amelia,"58,526",15.9,76.4,20.5,2.9,0.6


In [102]:
final_combined_dataset = pd.merge(left=town_closest_distances_filtered, right=county_socioeconomic, left_on='county', right_on='county')

In [103]:
final_combined_dataset.head()

,Town,latitude,longitude,closest_solid_waste,closest_renewable_energy,closest_petroleum_release,closest_pollution_reports,full_address,county,median income,bachelors degree,white,black,hispanic,asian
0,Abingdon town,36.71,-81.97,0.294818,52.129728,0.042871,0.460100,"162, A Street Southeast, Abingdon, Washington ...",Washington,"46,262",22.5,96.7,1.5,1.6,0.5
1,Damascus town,36.63,-81.79,11.642139,44.721626,0.332997,0.383418,"Damascus Middle School, Brook Street, Damascus...",Washington,"46,262",22.5,96.7,1.5,1.6,0.5
2,Emory,36.78,-81.83,6.305761,43.418193,0.429523,1.740788,"Linden Street, Emory, Washington County, Virgi...",Washington,"46,262",22.5,96.7,1.5,1.6,0.5
3,Glade Spring town,36.79,-81.77,4.441863,40.019822,0.092184,0.282392,"104, Highland Avenue, Glade Spring, Washington...",Washington,"46,262",22.5,96.7,1.5,1.6,0.5
4,Meadow View,36.76,-81.87,6.478409,45.888807,0.372990,0.332494,"13214, Summit Drive, Meadowview, Washington Co...",Washington,"46,262",22.5,96.7,1.5,1.6,0.5


In [104]:
final_combined_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 498 entries, 0 to 497
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Town                       498 non-null    object 
 1   latitude                   498 non-null    float64
 2   longitude                  498 non-null    float64
 3   closest_solid_waste        498 non-null    float64
 4   closest_renewable_energy   498 non-null    float64
 5   closest_petroleum_release  498 non-null    float64
 6   closest_pollution_reports  498 non-null    float64
 7   full_address               498 non-null    object 
 8   county                     498 non-null    object 
 9   median income              498 non-null    object 
 10  bachelors degree           498 non-null    float64
 11  white                      498 non-null    float64
 12  black                      498 non-null    float64
 13  hispanic                   498 non-null    float64

In [109]:
final_combined_dataset["median income"] = final_combined_dataset["median income"].apply(lambda x : float(x.replace(",", "")))

In [111]:
final_combined_dataset.to_pickle("final_combined_dataset.pkl")